# Basic scraping for Guardian discussions u

In [1]:
! ./.g


[master 159945f] web
 12 files changed, 2720 insertions(+), 498 deletions(-)
 rename .ipynb_checkpoints/{Untitled-checkpoint.ipynb => quad_forms-checkpoint.ipynb} (100%)
 create mode 100644 .ipynb_checkpoints/random_shapes-checkpoint.ipynb
 create mode 100644 None0000000.png
 create mode 100644 ejdxt.pkl
 create mode 100644 eyhx8.pkl
 create mode 100644 eytkx.pkl
 create mode 100644 femicides3.csv
 rewrite liberation_femicide.ipynb (78%)
 create mode 100644 links.txt
 rename Untitled.ipynb => quad_forms.ipynb (100%)
 create mode 100644 random_shapes.ipynb
Enumerating objects: 15, done.
Counting objects: 100% (15/15), done.
Delta compression using up to 4 threads
Compressing objects: 100% (11/11), done.
Writing objects: 100% (11/11), 1.43 MiB | 93.00 KiB/s, done.
Total 11 (delta 3), reused 0 (delta 0)
remote: Resolving deltas: 100% (3/3), completed with 2 local objects.
To https://github.com/macbuse/GUARDIAN.git
   07063a7..159945f  master -> master


In [59]:
with open('guardian comments scraper.ipynb','r') as fp:
    nb = fp.read()

In [60]:
import re

pp = re.compile("(https://www.the.*?)'")
lks = pp.findall(nb)

with open('links.txt','a') as fp:
    fp.write('\n'.join(lks))

In [1]:
import re 
import requests
import pickle
import json

In [34]:
article_url = 'https://www.theguardian.com/commentisfree/2020/aug/19/ditch-the-algorithm-generation-students-a-levels-politics'
article_url = 'https://www.theguardian.com/commentisfree/2020/aug/21/joe-biden-democrat-convention-nice-guy-trump-fight'
article_url = 'https://www.theguardian.com/commentisfree/2020/aug/23/the-cruel-exams-algorithm-has-laid-bare-the-unfairness-at-the-heart-of-our-schools'
article_url = 'https://www.theguardian.com/commentisfree/2020/sep/01/us-mainstream-right-vigilante-terror'
article_url = 'https://www.theguardian.com/commentisfree/2020/aug/31/politicians-covid-19-statistics-statisticians'
article_url = 'https://www.theguardian.com/commentisfree/2021/apr/14/boris-johnson-is-telling-scotland-that-the-union-is-no-longer-based-on-consent'

In [42]:
article_url = 'https://www.theguardian.com/commentisfree/2020/sep/29/rival-scientists-lockdowns-scientific-covid-19'
article_url = 'https://www.theguardian.com/commentisfree/2020/oct/08/johnson-and-sturgeon-will-need-a-better-philosophy-than-precaution-to-rid-us-of-covid'
article_url = 'https://www.theguardian.com/commentisfree/2020/oct/10/continual-local-lockdowns-answer-covid-control'
article_url = 'https://www.theguardian.com/commentisfree/2020/oct/11/the-rebel-scientists-cause-would-be-more-persuasive-if-it-werent-so-half-baked'

article_url = 'https://www.theguardian.com/commentisfree/2021/apr/03/undermining-the-astrazeneca-jab-is-a-dangerous-act-of-political-folly#comment-148440261'
article_url = 'https://www.theguardian.com/commentisfree/2021/apr/09/boris-johnson-brexit-belfast-violence-eu-good-friday-agreement'
article_url ='https://www.theguardian.com/commentisfree/2021/apr/02/culture-war-government-race-report'
article_url = 'https://www.theguardian.com/commentisfree/2021/mar/18/britain-sunny-vaccine-optimism-april-delivery-delay'
article_url ='https://www.theguardian.com/commentisfree/2021/apr/14/boris-johnson-is-telling-scotland-that-the-union-is-no-longer-based-on-consent'

In [36]:

key = my_keys.keys['guardian']

NameError: name 'my_keys' is not defined

In [43]:
#scrape the page and get the short url
short_url_pp =  re.compile('"shortUrlId":"(.*?)"')

r = requests.get(article_url)
mm = short_url_pp.search(r.text)
short_url = mm.group(1)

#the short url allows us to access the discussion
comments_url = 'https://api.nextgen.guardianapps.co.uk/discussion/%s.json'%short_url
params = {'orderBy' : 'newest',
          'pageSize' : 25,
          'displayThreaded' : 'true',
          'commentsClosed' : 'false',
          'page' : '',
          'maxResponses' : 50
         }

json_data =  []
start_page  = 1
last_page = None


for num_page in range(start_page, 10**4):
    print('getting comments page no.', num_page)
    params['page'] = num_page
    r = requests.get( comments_url, params=params)
    json_data.append( r.text)
    #The last page is a field in the json object
    #I don't need to read this as json I could do a regexp
    if num_page == start_page:
        json_dict = json.loads(r.text)
        last_page = json_dict['lastPage']
        print('there are %d pages'%last_page)
    if num_page == last_page : break
        

fn = '%s.pkl'%short_url.split('/')[-1]
print('archiving to %s'%fn)
with open(fn ,'wb') as fp:
    pickle.dump(json_data,fp)
    


getting comments page no. 1
there are 6 pages
getting comments page no. 2
getting comments page no. 3
getting comments page no. 4
getting comments page no. 5
getting comments page no. 6
archiving to h452h.pkl


## remember each page of comments has been stored separately

In [16]:
dd = json.loads(json_data[0])
len(json_data)

11

In [17]:
xx = json.loads(json_data[0])
xx.keys()

dict_keys(['postedCommentHtml', 'commentsHtml', 'commentCount', 'refreshStatus', 'lastPage', 'paginationHtml'])

In [136]:
print( xx['commentsHtml'][200:700])

 class="d-comment
            d-comment--top-level"
           id="comment-144422422"
           data-comment-id="144422422"
           data-comment-author="Hushing"
           data-comment-replies="0"
           data-comment-timestamp="2020-10-11T14:25:06.000+01:00"
           data-comment-author-id="12407494"
           data-comment-highlighted="false"
           itemscope itemtype="http://schema.org/Comment">

    <div class="d-comment__inner d-comment__inner--top-level">
        <div class="


In [18]:
pa = re.compile('data-comment-author="(.*?)"')
pid = re.compile('data-comment-author-id="(\d+)')
pcid = re.compile('\sid="comment-(.*?)"')
pp = re.compile('<li class="d-comment(.*?)>',re.DOTALL)

auths = []
md = []
for xx in json_data:
    hh = json.loads(xx)['commentsHtml']
    auths.extend(list( zip(pa.findall(hh), 
                           pid.findall(hh)
                          )))
    md.extend(pp.findall(hh))

auths = list(set(auths))
auths.sort(key=lambda x : x[0].lower())


In [102]:
#print( '\n'.join([x for x in md if 'pinkie' in x.lower()]))

https://discussion.theguardian.com/comment-permalink/144375739

In [8]:
auths = dict(auths)

In [9]:
comments_url

'https://api.nextgen.guardianapps.co.uk/discussion//p/hvpq8.json'

In [44]:
class Comment():
    pass

from bs4 import BeautifulSoup

def html2comments(x):
    tt = json.loads(x)
    soup = BeautifulSoup(tt['commentsHtml'],  "lxml")
    raw = [ comment.text for comment in  
            soup.findAll('div', {'class' : "d-comment__body"}) ]
    
    metadata = soup.findAll('span', {'class' : "d-comment__author"})
    
    
    auth = [ x.text.upper().strip() for x in metadata]
    auth_id = [ x.find('a')['href'] for x in metadata]
    #this was a pain to do really
    #the problem is blocked comments
    comment_id = [ x['id'][len('comment-'):] for x in soup.findAll('li')]
 
    
    ss = '##{}\nhttps://discussion.theguardian.com/comment-permalink/{}\n{}\n\n{}'
    return [ ss.format(a,b,d,c.strip())  
                   for a,b,d,c in zip(auth, comment_id, auth_id, raw)]
     
all_texts = [ html2comments(page) for page in json_data[:] ]



from itertools import chain
all_comments = list(chain.from_iterable(all_texts))
    

# Random data hacking

In [45]:
A = [x.split('\n')[0] for x in all_comments]
from collections import Counter
AN = Counter(A)
sorted([(x,y) for x,y in AN.items() if y > 4], key=lambda x : -x[1]) 

[('##FRAILEA', 40),
 ('##LYNNMA', 17),
 ('##COMPUTERSAYSPERHAPS', 17),
 ('##ETHOSDAIMON', 16),
 ('##DANNYNEDELKO', 13),
 ('##FIFI_EAT_MY_HAMSTER', 12),
 ('##PILRIG1954', 12),
 ('##SPORADICWIT', 11),
 ('##ATTILA9000', 11),
 ('##BOJOHARAM', 11),
 ('##ARTHUROPODD', 9),
 ('##KWAKWA', 9),
 ('##FREEDOMOFSPEECG', 9),
 ('##TUPPENCE_WORTH', 9),
 ('##CANARYATTHEWHARF', 8),
 ('##BISCUIT74', 8),
 ('##NICHOLIB', 8),
 ('##CADICYMWRAIG', 8),
 ('##CHRISPYTL', 8),
 ('##CURIA', 7),
 ('##BEWARETHEMYSTERONS', 7),
 ('##SHARK_SANDWICH', 7),
 ('##INARGUENDO', 7),
 ('##IMAGINARY_FIEND', 6),
 ('##NYAMUK', 6),
 ('##NOAMERICANO1', 6),
 ('##BRAVEHEARTABROAD', 6),
 ('##METALMAN71', 6),
 ('##ETPHONESHOME', 6),
 ('##EDITHCLUNGE', 6),
 ('##MARTINSILENUS', 5),
 ('##MRSPEX999', 5),
 ('##TONYNORTHWEST59', 5),
 ('##BATS2IN2THE2BELFRY', 5),
 ('##JACM111', 5),
 ('##MONKEYJOUK', 5),
 ('##DAVETHEWRAPPER', 5),
 ('##MRMOJOPIN', 5)]

In [149]:
! ./.g


[master b8c8a75] web
 2 files changed, 1457 insertions(+), 1317 deletions(-)
 rewrite guardian comments scraper.ipynb (76%)
Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 4 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 47.51 KiB | 84.00 KiB/s, done.
Total 4 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/macbuse/GUARDIAN.git
   d207214..b8c8a75  master -> master


In [150]:
print(all_comments[0])

##HUSHING
https://discussion.theguardian.com/comment-permalink/144422422
https://profile.theguardian.com/user/id/12407494

Any proposal should be clear in its conception, its aims, and what outcomes it would take the person proposing to reconsider.
What facts are you drawing on?What facts are you dismissing?What preconceptions are you making?What assumptions are you relying on?What do you aim to achieve? Why?How does what you propose achieve this?What do you aim to avoid? Why?How does what you propose avoid this?What answers to serious (non-strawman) criticisms do you have?What would you consider to be a sign that your proposal had failed?
Anything that doesn't answer these questions is performative bullshit.


In [46]:
all_comments.sort(key=lambda x : -len(x) )
print( '\n\n'.join(all_comments[:20]) )

##HAIGIN88
https://discussion.theguardian.com/comment-permalink/148638710
https://profile.theguardian.com/user/id/2836106

Not only did almost all Leavers demand that the June 2016 EU vote was recognised but so many Remainers too. It was just bad taste and not British to, in effect, claim that a one-off was a two-legged affair and that pissed off many a Remainer-voter as well. That so many went to the mat to defend that vote but some will wave through a re-run of a referendum from just a year and 10 months earlier? Crazy talk. One shouldn't shit on the 2014 'No, Thanks' vote as one shouldn't befoul the 2016 Leave vote.
Britons decide what happens to Britain: that was the Brexit theory of the case. If there was another one and it was a Yes vote then the rest of Britain would be turned into a de facto one-party (Tory) state, not by Westminster, not by Brussels but by a state north of the country. Not only that but such a result would be achieved with the rest of Britain being able to vot

In [18]:
#https://towardsdatascience.com/sentimental-analysis-using-vader-a3415fef7664

import nltk
#nltk.download('vader_lexicon')

from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/gregmcshane/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [22]:
a = '''My views haven't changed. Leaving the EU will be of no benefit to anyone except those who hold money and power along with certain corporates and a smidgen of medium to small businesses.
How else can it be when you are shutting shop to the most developed nations, the richest nations in the world, who are either in the EU or have market agreements with the EU. Even the emerging markets of China and India along with other developing economies are aspiring to develop trading agreements with the EU, the second largest economy in the world. Any deal with the UK will be secondary and will depend upon their agreement with the EU.
Boris Johnson once said "f**k business". I take this as meaning there is something in it for them and they don't really care if businesses find it difficulA
t to cope.
I can see ministers and conservative MPs queuing to fill their pockets by acting as advisors for potential trade deals whilst the rest of the country are left struggling trying to rebuild their lives, especially following the pandemic. The same ministers will probably be looking to lower all kinds of standards from food and welfare to safety at work to improve their opportunities.
When it comes to brinkmanship, I can only view this as getting away with whatever they can. I do not believe it is anything to do with negotiating an agreement that will be to the benefit of the UK or the EU as a whole, I don't believe our ministers are capable of doing that.'''

In [24]:
sid.polarity_scores(a)
sid.sentiment_valence(a)

TypeError: sentiment_valence() missing 4 required positional arguments: 'sentitext', 'item', 'i', and 'sentiments'

In [29]:
target = 'decolon'
target = 'science'
target = 'boris'


xx = [x for x in all_comments if target.lower() in x.lower() and  len(x) > 20]
yy = [x for x in all_comments if '#pin' in x.lower() and 2000 > len(x) > 20]

#print('\n\n'.join( ['\n\n'.join(x) for x in zip(xx,yy)])  )


#print( '\n\n'.join(xx).replace(target, ) )
pxx = re.compile(target, re.I)
print( pxx.sub('**' + target.upper() + '**', 
              '\n\n'.join(xx)) )
               
#print( '\n\n'.join(xx) )


##FLETCHISBACK
https://discussion.theguardian.com/comment-permalink/148427412
https://profile.theguardian.com/user/id/15531940

I never understand why the left think they are blameless in the culture wars, that this is something being done to them. 
We had article after article traducing Winston Churchill, before the statue was vandalised in the BLM protests. Then article after article saying that was perfectly fine. Then when **BORIS**, spying a political opportunity mentions it, knowing he is revered figure for many, suddenly **BORIS** is starting a culture war. No he didn't start it, but he is capitalising on it. He is saying to a sizable demographic - look the left hate your heroes, your values, and let's face it you. Now the left may well be right, Churchill may have been awful through and through. But it is interesting that concerns over policing in the USA leads to attacks on British culture. Wouldn't it be better that we had all these articles about the utterly toothless and ca

36